### Introdcution

This is a notebook for discussing the future directions of this project.
Please be notified, this is **_NOT_** the feature **_NOR_** the model used for the IPSN 2022 VMA paper.
The aim of this paper is becaues I envision the future of sensory-based Human Activity Recognition consists of encoder-decoder structures. The representation learning for all the down-stream task will be the near future. In the VMA paper, we utilize the hand-crafted feature and concatenate them together, which is quite limited. Also, the adoption of different segmentation technique make it qutie annoying in the fusion step. In this way, I want to explore, can we leverage physcial knowledge to conduct some feature extraction and them fit into the VMA framework?

This note book is different with in `demo_paper.ipynb` in (but not limited to) following points:
- It dosen't adpot hand-crafted feature. It's based on feature extrator network. This notebook may be kept updating, so the backbone is not fixed as I'm writing now.
- The vibration signal is processed with sliding window segementation at the beginning. Therefore, you dont't need all these MATLAB-based fusion codes. You can easliy do the confidence thresholding and pseudo-labeled data selection with numpy indexing.
- There is not any spatial analysis-based model. I just code one model for all activities.
- This notebook is highly possible to be unfinished/bad. I don't have too much time to build a very powerful model, that's not the contribution of the project. You may see the current model achieves a 0% accuracy. I just want to use this notebook to show more possibilities.

In [1]:
'''
Coded by Kevin Hu.
Contact: www.kevin-hu.com
'''
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from utils import *
import matplotlib.pyplot as plt
import numpy as np

In [2]:
batch_size = 256
num_epoch = 100
l_r = 0.001
window_size = int(6500*1.5)
verbose = 1

In [3]:
train_data = read_mat('./raw_data/person_1_location_1.mat')
inter_data = read_mat('./raw_data/person_1_location_2.mat')
test_data = read_mat('./raw_data/person_2_location_2.mat')

In [4]:
train_imu, train_vib =  prepare_tf_data(train_data, window_size, batch_size)
inter_imu, inter_vib =  prepare_tf_data(inter_data, window_size, batch_size)
test_imu, test_vib =  prepare_tf_data(test_data, window_size, batch_size)

In [6]:
loss_i = keras.losses.CategoricalCrossentropy(from_logits=True)
loss_v = keras.losses.CategoricalCrossentropy(from_logits=True)

opt_i= keras.optimizers.Adam(learning_rate=l_r)
opt_v= keras.optimizers.Adam(learning_rate=l_r)

train_acc_metric = keras.metrics.CategoricalAccuracy()
val_acc_metric = keras.metrics.CategoricalAccuracy()

In [7]:
share_input = layers.Input(shape=(128,))
s_1 = layers.Dense(64, activation="elu", name = 'share_1')(share_input)
s_1 = layers.Dropout(0.5)(s_1)
s_2 = layers.Dense(32, activation="elu", name = 'share_2')(s_1)
s_2 = layers.Dropout(0.5)(s_2)
shared_layers = keras.Model(inputs=share_input, outputs=s_2, name="middle_model")

In [8]:
input_i = layers.Input((window_size, 6))
input_v = layers.Input((window_size, 4))

In [9]:
#imu_encoder
x_i = layers.Conv1D(64, 5, activation="elu")(input_i)
x_i = layers.BatchNormalization()(x_i)
x_i = layers.Conv1D(64, 5, activation="elu")(x_i)
x_i = layers.MaxPooling1D(pool_size=2)(x_i)
x_i = layers.Flatten()(x_i)
x_i = layers.BatchNormalization()(x_i)
x_i = layers.Dense(128, activation="elu")(x_i)

#vib_encoder
x_v = layers.Conv1D(64, 5, activation="elu")(input_v)
x_v = layers.BatchNormalization()(x_v)
x_v = layers.Conv1D(64, 5, activation="elu")(x_v)
x_v = layers.MaxPooling1D(pool_size=2)(x_v)
x_v = layers.Flatten()(x_v)
x_v = layers.BatchNormalization()(x_v)
x_v = layers.Dense(128, activation="elu")(x_v)

In [10]:
x_imu = shared_layers(x_i)
x_vib = shared_layers(x_v)

In [11]:
output_i = layers.Dense(10)(x_imu)
output_v = layers.Dense(10)(x_vib)

In [12]:
model_i = keras.Model(inputs=input_i, outputs=output_i, name="imu_model")
model_v = keras.Model(inputs=input_v, outputs=output_v, name="vib_model")

In [13]:
train_loss_i = list()
val_loss_i = list()
train_loss_v = list()
val_loss_v = list()

In [ ]:
for epoch in range(num_epoch):
    if verbose:
        print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train_i, y_batch_train_i) in enumerate(train_imu):
        y_batch_train_i = tf.keras.utils.to_categorical(y_batch_train_i, num_classes=10, dtype='float32')

        with tf.GradientTape() as tape:
            logits_i = model_i(x_batch_train_i, training=True)  # Logits for this minibatch
            loss_value_i = loss_i(y_batch_train_i, logits_i)
        grads_i = tape.gradient(loss_value_i, model_i.trainable_weights)
        opt_i.apply_gradients(zip(grads_i, model_i.trainable_weights))

    if verbose:
        print("Training loss of model_i at epoch %d: %.4f" % (epoch + 1, float(loss_value_i)))

    for x_batch_val_i, y_batch_val_i in inter_imu:
        y_batch_val_i = tf.keras.utils.to_categorical(y_batch_val_i, num_classes=10, dtype='float32')
        val_logits_i = model_i(x_batch_val_i, training=False)
        val_loss_value_i = loss_i(y_batch_val_i, val_logits_i)
    if verbose:
        print("Val loss of model_i at epoch %d: %.4f"% (epoch + 1, float(val_loss_value_i)))
    train_loss_i.append(float(loss_value_i))
    val_loss_i.append(float(val_loss_value_i))

    for step, (x_batch_train_v, y_batch_train_v) in enumerate(train_vib):
        y_batch_train_v = tf.keras.utils.to_categorical(y_batch_train_v, num_classes=10, dtype='float32')

        with tf.GradientTape() as tape:
            logits_v = model_v(x_batch_train_v, training=True)  # Logits for this minibatch
            loss_value_v = loss_v(y_batch_train_v, logits_v)
        grads_v = tape.gradient(loss_value_v, model_v.trainable_weights)
        opt_v.apply_gradients(zip(grads_v, model_v.trainable_weights))
    if verbose:
        print("Training loss of model_v at epoch %d: %.4f"% (epoch + 1, float(loss_value_v)))

    for x_batch_val_v, y_batch_val_v in inter_vib:
        y_batch_val_v = tf.keras.utils.to_categorical(y_batch_val_v, num_classes=10, dtype='float32')
        val_logits_v = model_v(x_batch_val_v, training=False)
        val_loss_value_v = loss_v(y_batch_val_v, val_logits_v)
    if verbose:
        print("Val loss of model_v at epoch %d: %.4f"% (epoch + 1, float(val_loss_value_v)))
    train_loss_v.append(float(loss_value_v))
    val_loss_v.append(float(val_loss_value_v))


In [ ]:
plt.plot(train_loss_i)
plt.plot(val_loss_i)
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.title("IMU Loss")
plt.legend(['Train Loss', 'Val Loss'])
plt.ylim([0, 4])
plt.show()

plt.plot(train_loss_v)
plt.plot(val_loss_v)
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.title("Vibration Loss")
plt.legend(['Train Loss', 'Val Loss'])
plt.ylim([0, 4])
plt.show()

In [19]:
tf_imu, tf_vib =  prepare_tf_data(inter_data, window_size, 1)
X_inter_imu, y_inter_imu = prepare_np_data(tf_imu, window_size, 6)
X_inter_vib, y_inter_vib = prepare_np_data(tf_vib, window_size, 4)

In [20]:
vib_conf_mat = np.zeros((tf_vib.cardinality().numpy(),10))
imu_conf_mat = np.zeros((tf_imu.cardinality().numpy(),10))

In [21]:
layer = tf.keras.layers.Activation('softmax')

In [23]:
val_acc_metric.reset_states()
for idx, (x_batch_val_v, y_batch_val_v) in enumerate(tf_vib):
        y_batch_val_v = tf.keras.utils.to_categorical(y_batch_val_v, num_classes=10, dtype='float32')
        val_logits_v = model_v(x_batch_val_v, training=False)
        output = layer(val_logits_v)
        pred_logit = output.numpy()
        vib_conf_mat[idx] = pred_logit
        val_acc_metric.update_state(y_batch_val_v , val_logits_v)
        # val_acc_metric.reset_states()
        # break

val_acc_v = val_acc_metric.result()
print("Vibration train acc: %.4f" % (float(val_acc_v),))

Vibration train acc: 0.2629


In [28]:
val_acc_metric.reset_states()
for x_batch_val_i, y_batch_val_i in tf_imu:
        y_batch_val_i = tf.keras.utils.to_categorical(y_batch_val_i, num_classes=10, dtype='float32')
        val_logits_i = model_i(x_batch_val_i, training=False) 
        output = layer(val_logits_i)
        pred_logit = output.numpy()
        imu_conf_mat[idx] = pred_logit
        val_acc_metric.update_state(y_batch_val_i , val_logits_i)
        # break

val_acc_i = val_acc_metric.result()
print("IMU train acc: %.4f" % (float(val_acc_i),))
val_acc_metric.reset_states()

IMU train acc: 0.3079


Well, not so good right now...